## Week 3 Day 4 Morning - More Graphs

In [ ]:
%run "boaz_utils.ipynb"

Recall: mathematically, a graph is a set $V$ of __vertices__ and a set $E$ of pairs of these vertices which is known as the set of __edges__. We say that a vertex $u\in V$ is connected to $v\in V$ if the pair $(u,v)$ is in $E$.

A graph where $(u,v)$ is an edge if and only if $(v,u)$ is also an edge is known as an __undirected__ graph. Undirected graphs form an important special case. Otherwise if this is not promised to be the case, we say the graph is __directed__.

Sometimes the edges (or vertices) of the graph are labeled by a number, which we call a __weight__. For example in the case of the road network, we might label every road segment with the length of that segment in kilometers.

There are two main representations for graphs. We can always assume the vertices are simply identified by the numbers $0$ to $n-1$ for some $n$. 

The __adjacency list representation__ is an array $L$ where $L[i]$ is the list of all neighbors of the vertex $i$ (i.e., all $j$ such that $(i,j)\in E$)

The __adjacency matrix representation__ is an $n\times n$ two-dimensional array $M$ (i.e., matrix) such that $M[i][j]$ equals $1$ if $j$ is a neighbor of $i$ and equals $0$ otherwise.

## Example of a graph: the 2-dimensional grid

In [ ]:
def grid_neighbors(i,j,n):
    if i==n-1 and j== n-1: return []
    if i==n-1:
        return [i*n+j+1]
    if j==n-1:
        return [(i+1)*n+j]
    return [n*i+((j+1) % n), n*((i+1) % n)+j] 

In [ ]:
n = 5
grid = [ grid_neighbors(i,j,n) for i in range(n) for j in range(n)  ]

In [ ]:
draw_graph(grid,'grid_layout')

# Basic graph functions

In [ ]:
def neighbors(G,u):
    return G[u]

def isedge(G,u,v):
    for i in G[u]:
        if i == v:
            return True
    return False

def vertices(G):
    return list(range(len(G)))

def addedge(G,i,j):
    if not isedge(G, i, j):
        G[i].append(j)
        
def emptygraph(n):
    G = []
    for i in range(n):
        G.append([])
    return G

In [ ]:
neighbors(cycle,0)

In [ ]:
isedge(cycle,4,7)

In [ ]:
vertices(cycle)

__Question:__ Write function `list2matrix(G)` that convertes a graph `G` in adj list format to adj matrix format where `M[i][j]` equals `1`/`0` based on whether `i` is neighbor of `j`

In [ ]:
def zeros(n): return [0]*n
def printmatrix(M):
    for L in M:
        print(L)
        
def list2matrix(G):
    n = len(vertices(G))
    M = []
    for i in range(n):
        M.append(zeros(n))
    for i in range(n):
        for j in range(n):
            if isedge(G,i,j):
                M[i][j] = 1
    return M

In [ ]:
G = [[3,1],[0,2],[1,3],[2,0]]
M = list2matrix(G)
printmatrix(M)

In [ ]:
G = [[1,4],[0,2],[1,3],[2,4],[3,0]]
printmatrix(list2matrix(G))

## Example: Make graph undirected

__Exercise:__ Write a function `undir` that takes a graph `G` and outputs a graph `_G` that such that for every `i,j` the edge `i->j` is in `G` if and only if both  `j->i`  and `i->j` are in `_G`.

In [ ]:
def neighbors(G,u):
    return G[u]

def isedge(G,u,v):
    return v in neighbors(G,u)

def vertices(G):
    return list(range(len(G)))

def addedge(G,i,j): 
    if not j in G[i]:
        G[i].append(j)
        
def emptygraph(n):
    L = []
    for i in range(n):
        L.append([])
    return L

In [ ]:
def undirect(G):
    U = emptygraph(len(G))
    for u in vertices(G):
        for v in neighbors(G, u):
            addedge(U, u, v)
            addedge(U, v, u)
    return U

In [ ]:
G = [[1],[2],[0]]
undir(G)

In [ ]:
draw_graph(undir(G))

# Graph connectivity

Given $i,j$ and a graph $G$: find out if $i$ has a path to $j$ (perhaps indirectly) in the graph

Here is a natural suggestion for a recursive algorithm:

$connected(i,j,G)$ is True if $i$ is a neighbor of $j$, and otherwise it is True if there is some neighbor $k$ of $i$ such that $k$ is connected to $j$. 

Let's code it up try to see what happens:

In [ ]:
def connected(source, target, G):
    print(".",end="")
    if source == target:
        return True
    else:
        for k in neighbors(G, source):
            if connected(k, target, G):
                return True
        return False

In [ ]:
G = undir([[1],[2],[3],[4],[]])
draw_graph(G)

In [ ]:
connected(0,1,G)

In [ ]:
connected(0,2,G)

In [ ]:
connected(0,3,G)

The problem is that we are getting into an infinite loop! 
We can fix this by remembering which vertices we visited.

In [ ]:
def connected(source, target, G, visited = []):
    if not (source in visited):
        visited.append(source)
    if source == target:
        return True
    else:
        #print('variables are now')
        #print(str(source) + ' ' + str(target) + ' ' + str(visited))
        for k in neighbors(G, source):
            if not (k in visited):
                visited.append(k)
                if connected(k, target, G, visited):
                    return True
        return False
    

In [ ]:
G = undir([[1],[2],[3],[4],[]])
draw_graph(G)

In [ ]:
print(connected(0, 1, G, []))
#print('next one')
print(connected(0, 2, G, []))

In [ ]:
G = undir([[1],[2],[0],[]])
draw_graph(G)

In [ ]:
print (connected(0,1,G) , connected(0,3,G))

In [ ]:
def make_grid(n): # return a n by n grid with an isolated vertex
    G = emptygraph(n*n)
    for i in range(n):
        for j in range(n):
            v = i*n+j
            if i<n-1: addedge(G,v,(i+1)*n+j)
            if j<n-1: addedge(G,v,i*n+j+1)
    G = undir(G)
    return G
 
def grid_input(n):
    return (0,n*n,make_grid(n)+[[]])

In [ ]:
(s,t,G) = grid_input(5)
draw_graph(G, "grid_layout")

In [ ]:
connected(0,1,G)

## Can speed up by changing how 'visited' works. Make it a length n list, where visited[u] is True if we ever visited vertex u and False otherwise

In [ ]:
# DFS = Depth First Search
def search(u, G, visited):
    visited[u] = True
    for v in neighbors(G, u):
        if not visited[v]:
            search(v, G, visited)
    
def dfs(source, target, G):
    visited = [False]*len(G)
    search(source, G, visited)
    return visited[target]

In [ ]:
G = undir([[1],[2],[0],[]])
draw_graph(G)

In [ ]:
print (dfs(0,1,G) , dfs(0,3,G))

## Exercise: think about the order DFS visits vertices in the graph below

In [ ]:
G = undir([[1,4],[2],[3],[2],[0]])
draw_graph(G)